In [1]:
import pandas
import os 
import torch
import glob

file_path =  "/home/link-lap-24/Downloads/jay_sir/passing.csv"
csvFile = pandas.read_csv(file_path)
print(csvFile)

     Type          Tags                                         Source Url  \
0   train  passing_shot  https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL...   
1   train  passing_shot  https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL...   
2   train  passing_shot  https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL...   
3   train  passing_shot  https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL...   
4   train  passing_shot  https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL...   
..    ...           ...                                                ...   
63  train  passing_shot  https://ai-studio-new.s3-ap-south-1.amazonaws....   
64  train  passing_shot  https://ai-studio-new.s3-ap-south-1.amazonaws....   
65  train  passing_shot  https://ai-studio-new.s3-ap-south-1.amazonaws....   
66  train  passing_shot  https://ai-studio-new.s3-ap-south-1.amazonaws....   
67  train  passing_shot  https://ai-studio-new.s3-ap-south-1.amazonaws....   

    startDuration  endDuration  
0             NaN          NaN

In [3]:
csvFile

,Type,Tags,Source Url,startDuration,endDuration
0,train,passing_shot,https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL...,NaN,NaN
1,train,passing_shot,https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL...,0.199138,13.052586
2,train,passing_shot,https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL...,1.289655,16.044828
3,train,passing_shot,https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL...,0.000000,5.732759
4,train,passing_shot,https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL...,1.696552,10.634483
...,...,...,...,...,...
63,train,passing_shot,https://ai-studio-new.s3-ap-south-1.amazonaws....,NaN,NaN
64,train,passing_shot,https://ai-studio-new.s3-ap-south-1.amazonaws....,NaN,NaN
65,train,passing_shot,https://ai-studio-new.s3-ap-south-1.amazonaws....,NaN,NaN
66,train,passing_shot,https://ai-studio-new.s3-ap-south-1.amazonaws....,NaN,NaN


In [2]:
urls = csvFile['Source Url']
# print(urls)

In [6]:
input_frames = "/home/link-lap-24/Downloads/jay_sir/input_frames/"
working_dir = "/home/link-lap-24/Downloads/jay_sir/working_dir/"

In [3]:
url3 = "http://localhost:8101/detect"  # 8120

In [4]:
# model = torch.hub.load('ultralytics/yolov5', 'custom', path="/home/link-lap-24/Downloads/jay_sir/sTyw2BYVM63VHl1.pt" 

In [7]:
import os, shutil, natsort,requests, cv2, traceback

def perform_detection(image_path, detection_api):
    payload = {"image_path": image_path}
    req = requests.post(url=detection_api, json=payload)
    result = req.json()['result']
    return result

for inx,url in enumerate(urls[30:]):
    print(inx, url)
    vid = url.split("/")[-1].split(".")[0]
    print("vid", vid)
    
    if os.path.exists(input_frames):
        shutil.rmtree(input_frames)
    os.mkdir(input_frames)
    
    #frame extract
    
    cmd = f"""ffmpeg -loglevel quiet -i {url} {input_frames}%d.jpg"""
    os.system(cmd)
    frames = natsort.natsorted(os.listdir(input_frames))
    
    for img in frames:
        image = cv2.imread(input_frames+img)
        frame_path = input_frames + img
        result = perform_detection(frame_path, url3)
        print(img ,result)
        
        count = 0
        for res in result:
            coordinates = res['coordinates']
            label = res['label']
            score = res['score']
            print()
            
            x1,y1,x2,y2 = coordinates
            x1 -= int(1920*0.5/100)
            y1 -= int(1920*0.5/100)
            x2 += int(1920*0.5/100)
            y2 += int(1920*0.5/100)
            
            crop_img = image[int(y1):int(y2), int(x1):int(x2)]
            
            out_dir = working_dir + label + "/"
            if not os.path.exists(out_dir):
                os.mkdir(out_dir)
            out_img_path = out_dir + vid + "_" + str(count) + "_" + img
            
            try:
                cv2.imwrite(out_img_path, crop_img) 
            except:
                print(traceback.print_exc())
            count += 1

0 https://dwapv64icf8j2.cloudfront.net/T4CuLKPZL/0PecwlHHL/1670829854_2903643/1670829738_174856.mp4
vid 1670829738_174856
1.jpg [{'coordinates': [1036, 402, 1069, 432], 'label': 'blur_ball', 'score': 0.79}, {'coordinates': [1037, 323, 1064, 342], 'label': 'blur_ball', 'score': 0.3}]


2.jpg [{'coordinates': [1036, 407, 1069, 438], 'label': 'blur_ball', 'score': 0.78}, {'coordinates': [1037, 323, 1064, 342], 'label': 'blur_ball', 'score': 0.3}]


3.jpg [{'coordinates': [1035, 411, 1068, 443], 'label': 'blur_ball', 'score': 0.8}]

4.jpg [{'coordinates': [1035, 417, 1069, 450], 'label': 'blur_ball', 'score': 0.8}]

5.jpg [{'coordinates': [1036, 430, 1067, 462], 'label': 'blur_ball', 'score': 0.8}, {'coordinates': [1037, 323, 1064, 343], 'label': 'blur_ball', 'score': 0.4}]


6.jpg [{'coordinates': [1035, 437, 1067, 469], 'label': 'blur_ball', 'score': 0.81}]

7.jpg [{'coordinates': [1036, 445, 1066, 477], 'label': 'blur_ball', 'score': 0.81}]

8.jpg [{'coordinates': [1036, 453, 1065, 484]

94.jpg [{'coordinates': [711, 469, 738, 498], 'label': 'blur_ball', 'score': 0.79}]

95.jpg [{'coordinates': [709, 473, 737, 502], 'label': 'blur_ball', 'score': 0.78}]

96.jpg [{'coordinates': [710, 477, 736, 505], 'label': 'blur_ball', 'score': 0.76}]

97.jpg [{'coordinates': [708, 478, 736, 507], 'label': 'blur_ball', 'score': 0.77}]

98.jpg [{'coordinates': [708, 481, 736, 512], 'label': 'blur_ball', 'score': 0.8}]

99.jpg [{'coordinates': [706, 485, 735, 515], 'label': 'blur_ball', 'score': 0.8}]

100.jpg [{'coordinates': [704, 492, 733, 523], 'label': 'blur_ball', 'score': 0.8}]

101.jpg [{'coordinates': [703, 496, 734, 528], 'label': 'blur_ball', 'score': 0.8}]

102.jpg [{'coordinates': [704, 501, 733, 530], 'label': 'blur_ball', 'score': 0.75}]

103.jpg [{'coordinates': [702, 503, 731, 534], 'label': 'blur_ball', 'score': 0.79}]

104.jpg [{'coordinates': [702, 503, 731, 532], 'label': 'blur_ball', 'score': 0.78}]

105.jpg [{'coordinates': [700, 501, 730, 531], 'label': 'blur_ba

177.jpg [{'coordinates': [1250, 795, 1292, 831], 'label': 'blur_ball', 'score': 0.78}]

178.jpg [{'coordinates': [1253, 792, 1293, 827], 'label': 'blur_ball', 'score': 0.79}]

179.jpg [{'coordinates': [1256, 790, 1294, 825], 'label': 'blur_ball', 'score': 0.81}]

180.jpg [{'coordinates': [1260, 787, 1296, 821], 'label': 'blur_ball', 'score': 0.81}]

181.jpg [{'coordinates': [1264, 779, 1302, 814], 'label': 'blur_ball', 'score': 0.79}]

182.jpg [{'coordinates': [1269, 778, 1303, 812], 'label': 'blur_ball', 'score': 0.79}]

183.jpg [{'coordinates': [1272, 776, 1305, 809], 'label': 'blur_ball', 'score': 0.8}]

184.jpg [{'coordinates': [1275, 775, 1306, 806], 'label': 'blur_ball', 'score': 0.78}]

185.jpg [{'coordinates': [1278, 774, 1307, 803], 'label': 'blur_ball', 'score': 0.75}]

186.jpg [{'coordinates': [1284, 772, 1313, 798], 'label': 'blur_ball', 'score': 0.37}]

187.jpg [{'coordinates': [1286, 770, 1316, 798], 'label': 'blur_ball', 'score': 0.65}]

188.jpg []
189.jpg []
190.jpg []


467.jpg [{'coordinates': [1250, 392, 1282, 424], 'label': 'blur_ball', 'score': 0.8}, {'coordinates': [959, 339, 983, 364], 'label': 'blur_ball', 'score': 0.5}]


468.jpg [{'coordinates': [1250, 385, 1284, 415], 'label': 'blur_ball', 'score': 0.82}, {'coordinates': [959, 339, 983, 363], 'label': 'blur_ball', 'score': 0.31}]


469.jpg [{'coordinates': [1250, 382, 1283, 411], 'label': 'blur_ball', 'score': 0.82}, {'coordinates': [959, 338, 984, 363], 'label': 'blur_ball', 'score': 0.49}]


470.jpg [{'coordinates': [1250, 381, 1283, 408], 'label': 'blur_ball', 'score': 0.81}, {'coordinates': [1274, 621, 1303, 660], 'label': 'blur_ball', 'score': 0.26}]


471.jpg [{'coordinates': [1249, 378, 1283, 405], 'label': 'blur_ball', 'score': 0.8}, {'coordinates': [959, 337, 984, 362], 'label': 'blur_ball', 'score': 0.36}]


472.jpg [{'coordinates': [1248, 377, 1282, 403], 'label': 'blur_ball', 'score': 0.81}]

473.jpg [{'coordinates': [1249, 374, 1281, 398], 'label': 'blur_ball', 'score': 0.79}]



553.jpg [{'coordinates': [752, 178, 782, 208], 'label': 'blur_ball', 'score': 0.82}, {'coordinates': [971, 334, 997, 358], 'label': 'blur_ball', 'score': 0.54}, {'coordinates': [1046, 333, 1077, 359], 'label': 'blur_ball', 'score': 0.41}]



554.jpg [{'coordinates': [747, 173, 778, 202], 'label': 'blur_ball', 'score': 0.81}, {'coordinates': [971, 334, 997, 358], 'label': 'blur_ball', 'score': 0.47}, {'coordinates': [1046, 333, 1077, 359], 'label': 'blur_ball', 'score': 0.34}]



555.jpg [{'coordinates': [745, 170, 776, 199], 'label': 'blur_ball', 'score': 0.8}, {'coordinates': [971, 334, 997, 358], 'label': 'blur_ball', 'score': 0.5}]


556.jpg [{'coordinates': [741, 168, 771, 195], 'label': 'blur_ball', 'score': 0.78}]

557.jpg [{'coordinates': [740, 166, 767, 191], 'label': 'blur_ball', 'score': 0.7}, {'coordinates': [1046, 333, 1077, 359], 'label': 'blur_ball', 'score': 0.43}, {'coordinates': [970, 334, 997, 358], 'label': 'blur_ball', 'score': 0.3}]



558.jpg [{'coordinates': [735

...

Traceback (most recent call last):
  File "/tmp/ipykernel_69249/197333720.py", line 51, in <cell line: 9>
    cv2.imwrite(out_img_path, crop_img)
cv2.error: OpenCV(4.5.5) /io/opencv/modules/imgcodecs/src/loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'



97.jpg [{'coordinates': [1058, 13, 1079, 33], 'label': 'blur_ball', 'score': 0.74}, {'coordinates': [1072, 154, 1097, 175], 'label': 'blur_ball', 'score': 0.69}]


98.jpg [{'coordinates': [1057, 20, 1077, 38], 'label': 'blur_ball', 'score': 0.63}]

99.jpg []
100.jpg []
101.jpg []
102.jpg []
103.jpg []
104.jpg []
105.jpg [{'coordinates': [1011, 139, 1036, 161], 'label': 'blur_ball', 'score': 0.28}]

106.jpg [{'coordinates': [971, 91, 1012, 125], 'label': 'blur_ball', 'score': 0.73}]

107.jpg [{'coordinates': [944, 110, 985, 145], 'label': 'blur_ball', 'score': 0.58}]

108.jpg [{'coordinates': [917, 129, 961, 170], 'label': 'blur_ball', 'score': 0.52}]

109.jpg [{'coordinates': [890, 141, 939, 191], 'label': 'blur_ball', 'score': 0.85}]

110.jpg [{'coordinates': [866, 161, 918, 212], 'label': 'blur_ball', 'score': 0.86}]

111.jpg [{'coordinates': [844, 185, 892, 230], 'label': 'blur_ball', 'score': 0.85}]

112.jpg [{'coordinates': [822, 205, 870, 250], 'label': 'blur_ball', 'score': 0.87

195.jpg [{'coordinates': [1005, 157, 1048, 197], 'label': 'blur_ball', 'score': 0.83}]

196.jpg [{'coordinates': [1014, 160, 1056, 199], 'label': 'blur_ball', 'score': 0.81}]

197.jpg [{'coordinates': [1025, 165, 1064, 202], 'label': 'blur_ball', 'score': 0.81}]

198.jpg [{'coordinates': [1032, 168, 1073, 207], 'label': 'blur_ball', 'score': 0.84}]

199.jpg [{'coordinates': [1043, 174, 1082, 211], 'label': 'blur_ball', 'score': 0.83}]

200.jpg [{'coordinates': [1052, 177, 1092, 216], 'label': 'blur_ball', 'score': 0.84}]

201.jpg [{'coordinates': [1060, 183, 1099, 220], 'label': 'blur_ball', 'score': 0.81}]

202.jpg [{'coordinates': [1068, 187, 1108, 225], 'label': 'blur_ball', 'score': 0.84}]

203.jpg [{'coordinates': [1076, 193, 1114, 228], 'label': 'blur_ball', 'score': 0.82}]

204.jpg [{'coordinates': [1083, 198, 1124, 235], 'label': 'blur_ball', 'score': 0.84}]

205.jpg [{'coordinates': [1091, 205, 1130, 240], 'label': 'blur_ball', 'score': 0.83}]

206.jpg [{'coordinates': [1098, 

Traceback (most recent call last):
  File "/tmp/ipykernel_69249/197333720.py", line 51, in <cell line: 9>
    cv2.imwrite(out_img_path, crop_img)
cv2.error: OpenCV(4.5.5) /io/opencv/modules/imgcodecs/src/loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'



262.jpg [{'coordinates': [1114, 376, 1163, 415], 'label': 'blur_ball', 'score': 0.86}]

263.jpg [{'coordinates': [1091, 392, 1147, 444], 'label': 'blur_ball', 'score': 0.85}]

264.jpg [{'coordinates': [1067, 425, 1118, 474], 'label': 'blur_ball', 'score': 0.82}]

265.jpg [{'coordinates': [1038, 454, 1098, 507], 'label': 'blur_ball', 'score': 0.85}]

266.jpg [{'coordinates': [1014, 483, 1069, 540], 'label': 'blur_ball', 'score': 0.85}]

267.jpg [{'coordinates': [1001, 516, 1040, 563], 'label': 'blur_ball', 'score': 0.85}]

268.jpg [{'coordinates': [966, 561, 996, 598], 'label': 'blur_ball', 'score': 0.77}]

269.jpg [{'coordinates': [947, 575, 983, 602], 'label': 'blur_ball', 'score': 0.78}]

270.jpg [{'coordinates': [923, 571, 967, 605], 'label': 'blur_ball', 'score': 0.27}]

271.jpg [{'coordinates': [904, 575, 947, 608], 'label': 'blur_ball', 'score': 0.75}]

272.jpg [{'coordinates': [889, 582, 926, 609], 'label': 'blur_ball', 'score': 0.28}]

273.jpg [{'coordinates': [873, 589, 907, 6